In [1]:
#!!!!WARNING!!! if you are using google colab, do not run couple code blocks together to prevent corrupting and related errors.

# Import the library
import sqlite3

# Connect to a database (or create it if it doesn't exist)
# This creates a file called 'crypto_portfolio.db' in your Colab environment
conn = sqlite3.connect('crypto_portfolio.db')

# Create a cursor object. This is how we execute SQL commands.
cur = conn.cursor()

# Define an SQL command as a Python string to create a table
create_table_sql = """
CREATE TABLE IF NOT EXISTS crypto_prices (
    id INTEGER PRIMARY KEY AUTOINCREMENT,  -- A unique ID for each row
    cryptocurrency TEXT NOT NULL,          -- Name of the crypto, e.g., 'bitcoin'
    price_usd REAL,                        -- The price, a real number (float)
    timestamp TEXT                         -- When we recorded the price
);
"""

# Execute the SQL command to create the table
cur.execute(create_table_sql)

# Commit the changes to save them to the database
conn.commit()

print("Database and table created successfully!")

Database and table created successfully!


In [5]:
import requests
import sqlite3
from datetime import datetime

# --- PART 1: EXTRACT DATA FROM API ---
url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
response = requests.get(url)
data = response.json()
price = data['bitcoin']['usd']
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S") # Formatted timestamp

# --- PART 2: LOAD DATA INTO SQL DATABASE ---
# Connect to the database we just created
conn = sqlite3.connect('crypto_portfolio.db')
cur = conn.cursor()

# Define an SQL command to INSERT a new record
# The (?) are placeholders for the values we will provide
insert_sql = """
INSERT INTO crypto_prices (cryptocurrency, price_usd, timestamp)
VALUES (?, ?, ?);
"""

# Execute the command, passing a tuple of values for the placeholders
# The order must match the column order: (cryptocurrency, price_usd, timestamp)
cur.execute(insert_sql, ('bitcoin', price, current_time))

# Commit the changes to SAVE the transaction
conn.commit()

# Close the connection. Good practice to avoid locking the file.
conn.close()

print("Data successfully inserted into the database!")

Data successfully inserted into the database!


In [8]:
# Reconnect to the database if your connection is closed
conn = sqlite3.connect('crypto_portfolio.db')
cur = conn.cursor()

# Define a SELECT query to get all data from the crypto_prices table
select_sql = "SELECT cryptocurrency, price_usd FROM crypto_prices;"

# Execute the query
cur.execute(select_sql)

# Fetch all the results of the query
results = cur.fetchall()

# Print the results
print("All records in the crypto_prices table:")
for row in results:
    print(row)

# Always close the connection when you're done
conn.close()

All records in the crypto_prices table:
('bitcoin', 110096.0)


In [9]:
import sqlite3
from datetime import datetime

# Connect to the DB
conn = sqlite3.connect('crypto_portfolio.db')
cur = conn.cursor()

# First, let's add another record for a different crypto to have more data
insert_sql = "INSERT INTO crypto_prices (cryptocurrency, price_usd, timestamp) VALUES (?, ?, ?);"
cur.execute(insert_sql, ('ethereum', 3000.0, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
conn.commit()
print("Added an Ethereum record.")

# Now, let's say the price of Ethereum changed. We need to UPDATE it.
update_sql = """
UPDATE crypto_prices
SET price_usd = 3500.0  -- The new value for the column
WHERE cryptocurrency = 'ethereum';  -- The condition: which row(s) to update
"""

cur.execute(update_sql)
conn.commit() # Remember to commit!
print("Updated Ethereum price.")

# Let's check our work by selecting all data
cur.execute("SELECT * FROM crypto_prices;")
results = cur.fetchall()
print("\nAll records after update:")
for row in results:
    print(row)

conn.close()

Added an Ethereum record.
Updated Ethereum price.

All records after update:
(1, 'bitcoin', 110096.0, '2025-08-26 11:13:18')
(2, 'ethereum', 3500.0, '2025-08-26 11:23:47')


In [10]:
import sqlite3

conn = sqlite3.connect('crypto_portfolio.db')
cur = conn.cursor()

# Let's say we want to delete all test records for 'ethereum'
# First, let's see what we're about to delete (SAFETY FIRST)
check_sql = "SELECT * FROM crypto_prices WHERE cryptocurrency = 'ethereum';"
cur.execute(check_sql)
to_delete = cur.fetchall()
print("Records to be deleted:")
for row in to_delete:
    print(row)

# Now, if we're sure, we delete them.
delete_sql = "DELETE FROM crypto_prices WHERE cryptocurrency = 'ethereum';"
cur.execute(delete_sql)
conn.commit() # Commit the deletion
print("\nEthereum records deleted.")

# Verify deletion
cur.execute("SELECT * FROM crypto_prices;")
results = cur.fetchall()
print("\nAll records after deletion:")
for row in results:
    print(row)

conn.close()

Records to be deleted:
(2, 'ethereum', 3500.0, '2025-08-26 11:23:47')

Ethereum records deleted.

All records after deletion:
(1, 'bitcoin', 110096.0, '2025-08-26 11:13:18')


In [11]:
import sqlite3
from datetime import datetime

# Connect to the DB
conn = sqlite3.connect('crypto_portfolio.db')
cur = conn.cursor()

# Now, let's say the price of bitcoin changed. We need to UPDATE it.
update_sql = """
UPDATE crypto_prices
SET price_usd = 1.0  -- The new value for the column
WHERE cryptocurrency = 'bitcoin';  -- The condition: which row(s) to update
"""

cur.execute(update_sql)
conn.commit() # Remember to commit!
print("Updated bitcoin price.")

# Let's check our work by selecting all data
cur.execute("SELECT * FROM crypto_prices;")
results = cur.fetchall()
print("\nAll records after update:")
for row in results:
    print(row)

conn.close()

Updated bitcoin price.

All records after update:
(1, 'bitcoin', 1.0, '2025-08-26 11:13:18')


In [16]:
#Complete ETL pipeline (Export, Transform, Load).
import sqlite3
import requests
from datetime import datetime

def create_database_connection():
    """Creates a connection to the SQLite database."""
    conn = sqlite3.connect('crypto_portfolio.db')
    return conn

def create_table(conn):
    """Creates the crypto_prices table if it doesn't exist."""
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS crypto_prices (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        cryptocurrency TEXT NOT NULL,
        price_usd REAL,
        timestamp TEXT
    );
    """
    cur = conn.cursor()
    cur.execute(create_table_sql)
    conn.commit() # Commit the table creation

def fetch_crypto_data():
    """Fetches the latest prices from the CoinGecko API."""
    url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin,ethereum&vs_currencies=usd"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an error for bad status codes
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"API Request failed: {e}")
        return None

def insert_data(conn, data):
    """Inserts the extracted data into the database."""
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cur = conn.cursor()
    insert_sql = "INSERT INTO crypto_prices (cryptocurrency, price_usd, timestamp) VALUES (?, ?, ?);"

    # Data is a dict: {'bitcoin': {'usd': 50000}, 'ethereum': {'usd': 3000}}
    for coin, price_data in data.items():
        price = price_data['usd']
        # Execute the insert for each cryptocurrency
        cur.execute(insert_sql, (coin, price, current_time))

    conn.commit() # Commit all inserts at once
    print(f"Inserted data for {list(data.keys())}")

def query_latest_data(conn):
    """Queries and displays the most recent entries for each cryptocurrency."""
    cur = conn.cursor()
    # This query gets the latest price for each crypto
    query_sql = """
    SELECT cryptocurrency, price_usd, timestamp
    FROM crypto_prices
    WHERE id IN (
        SELECT MAX(id)
        FROM crypto_prices
        GROUP BY cryptocurrency
    );
    """
    cur.execute(query_sql)
    results = cur.fetchall()

    print("\n--- Latest Prices in Database ---")
    for row in results:
        print(f"{row[0]}: ${row[1]:.2f} at {row[2]}")

# --- MAIN EXECUTION ---
# This is where the script actually runs
print("Starting ETL Pipeline...")

# 1. Create DB & Table
db_conn = create_database_connection()
create_table(db_conn)

# 2. Extract Data from API
crypto_data = fetch_crypto_data()

if crypto_data:
    # 3. Load Data into DB
    insert_data(db_conn, crypto_data)
    # 4. Query and Verify
    query_latest_data(db_conn)
else:
    print("Skipping insert due to API error.")

# 5. Close the connection
db_conn.close()
print("\nPipeline run complete.")

Starting ETL Pipeline...
Inserted data for ['bitcoin', 'ethereum']

--- Latest Prices in Database ---
bitcoin: $109682.00 at 2025-08-26 11:44:54
ethereum: $4418.19 at 2025-08-26 11:44:54

Pipeline run complete.
